In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-dataset-accenture/cleaned_ReactionTypes.csv
/kaggle/input/cleaned-dataset-accenture/cleaned_content4.csv
/kaggle/input/cleaned-dataset-accenture/cleaned_Reactions4.csv


In [2]:
reactions_df = pd.read_csv("/kaggle/input/cleaned-dataset-accenture/cleaned_Reactions4.csv")
rtypes_df = pd.read_csv("/kaggle/input/cleaned-dataset-accenture/cleaned_ReactionTypes.csv")
content_df = pd.read_csv("/kaggle/input/cleaned-dataset-accenture/cleaned_content4.csv", index_col=0)


In [3]:
rtypes_df.head()

,Type,Score
0,heart,60
1,want,70
2,disgust,0
3,hate,5
4,interested,30


In [4]:
content_df.head()

,Content ID,Type,Category
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,photo,Studying
1,9f737e0a-3cdd-4d29-9d24-753f4e3be810,photo,Healthy eating
2,230c4e4d-70c3-461d-b42c-ec09396efb3f,photo,Healthy eating
3,356fff80-da4d-4785-9f43-bc1261031dc6,photo,Technology
4,01ab84dd-6364-4236-abbb-3f237db77180,video,Food


In [5]:
reactions_content = reactions_df.merge(content_df, on="Content ID", suffixes=("_reaction","_content"))

reactions_content.head()

,Content ID,User ID,Type_reaction,Datetime,Type_content,Category
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,5d454588-283d-459d-915d-c48a2cb4c27f,disgust,2020-11-07 09:43:50,photo,Studying
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,92b87fa5-f271-43e0-af66-84fac21052e6,dislike,2021-06-17 12:22:51,photo,Studying
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,163daa38-8b77-48c9-9af6-37a6c1447ac2,scared,2021-04-18 05:13:58,photo,Studying
3,97522e57-d9ab-4bd6-97bf-c24d952602d2,34e8add9-0206-47fd-a501-037b994650a2,disgust,2021-01-06 19:13:01,photo,Studying
4,97522e57-d9ab-4bd6-97bf-c24d952602d2,9b6d35f9-5e15-4cd0-a8d7-b1f3340e02c4,interested,2020-08-23 12:25:58,photo,Studying


In [6]:
all_merge = reactions_content.merge(rtypes_df, left_on="Type_reaction", right_on="Type")

all_merge.head()

,Content ID,User ID,Type_reaction,Datetime,Type_content,Category,Type,Score
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,5d454588-283d-459d-915d-c48a2cb4c27f,disgust,2020-11-07 09:43:50,photo,Studying,disgust,0
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,34e8add9-0206-47fd-a501-037b994650a2,disgust,2021-01-06 19:13:01,photo,Studying,disgust,0
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,13c06e7e-833d-47eb-a790-5e09ccfd8d2c,disgust,2021-04-09 02:46:20,photo,Studying,disgust,0
3,9f737e0a-3cdd-4d29-9d24-753f4e3be810,8b49caad-bcc5-43de-bf40-34a66ff8805c,disgust,2021-03-28 21:15:26,photo,Healthy eating,disgust,0
4,230c4e4d-70c3-461d-b42c-ec09396efb3f,ef147ea5-9696-44d5-b6c2-a43f62fd8ce2,disgust,2020-08-04 05:40:33,photo,Healthy eating,disgust,0


In [7]:
# select relevant columns
rel_columns = all_merge[["Content ID", "Datetime", "Type_content", "Category", "Type", "Score"]]

group_cats = rel_columns.groupby("Category")["Score"].sum()

top_5 = group_cats.sort_values(ascending=False)[:5]

top_5

Category
Animals           74654
Science           70914
Healthy eating    69094
Technology        68429
Food              66089
Name: Score, dtype: int64

In [8]:
rel_columns.head()

,Content ID,Datetime,Type_content,Category,Type,Score
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,2020-11-07 09:43:50,photo,Studying,disgust,0
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,2021-01-06 19:13:01,photo,Studying,disgust,0
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,2021-04-09 02:46:20,photo,Studying,disgust,0
3,9f737e0a-3cdd-4d29-9d24-753f4e3be810,2021-03-28 21:15:26,photo,Healthy eating,disgust,0
4,230c4e4d-70c3-461d-b42c-ec09396efb3f,2020-08-04 05:40:33,photo,Healthy eating,disgust,0


In [9]:
rel_columns.rename(columns = {'Type_content':'Content Type', 'Type': 'Reaction Type'}, inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
rel_columns.head()

,Content ID,Datetime,Content Type,Category,Reaction Type,Score
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,2020-11-07 09:43:50,photo,Studying,disgust,0
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,2021-01-06 19:13:01,photo,Studying,disgust,0
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,2021-04-09 02:46:20,photo,Studying,disgust,0
3,9f737e0a-3cdd-4d29-9d24-753f4e3be810,2021-03-28 21:15:26,photo,Healthy eating,disgust,0
4,230c4e4d-70c3-461d-b42c-ec09396efb3f,2020-08-04 05:40:33,photo,Healthy eating,disgust,0


In [11]:
with pd.ExcelWriter("final_datasheet.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    rel_columns.to_excel(writer, sheet_name="Scores for Contents", index=False)
    top_5.to_excel(writer, sheet_name="Top 5 Categories")